In [11]:
import numpy as np
import matplotlib.pylab as plt
import math as math
from scipy.optimize import fmin
from iminuit import Minuit, describe, Struct
from scipy.spatial import distance
from scipy.sparse import vstack

import time

#import lichen as lch
import seaborn as sns

%matplotlib  notebook

In [26]:
# Add code to the following function so that it takes in two datasets, loops over one of them, and finds
# information about the nearest neighbors in the other dataset, based on a flag. 

def nn(data0,data1,r=None,nneighbors=None):
    
    ret = -1
    ret_list=[]
    if r is not None and nneighbors is not None:
        exit(-1)
        return ret
    elif r is not None and nneighbors is None:
        rsq = r*r
        print data0
        print '-----------'
        #for num0,num1 in zip(data0[0],data0[1]):
        for i,d in enumerate(data0.transpose()):
            
            count=0
            diffx=d[0]-data1[0]
            diffy=d[1]-data1[1]
            #diff= np.sqrt((num0-data1[0])**2 +(num1-data1[1])**2)
            #diff= np.sqrt(diffx*diffx + diffy*diffy)
            diff= diffx*diffx + diffy*diffy
            #if i>0:
            #    print '-------------------'
            #    print np.sqrt(diff)[0:10]
            #print diff
            #print '****************'
            count = len(diff[diff<rsq])
            ret_list.append(float(count)/(float(len(data1[0]))*r))
        ret_list = np.array(ret_list)
        #print ret_list
        #exit()
        return ret_list
    elif r is None and nneighbors is not None:
        for num0 in data0:
            diff = np.abs(num0 - data1)
            diff.sort()
            radius= diff[nneighbors-1]
            ret_list.append(1/radius)
        ret_list = np.array(ret_list)
        return ret_list
    return ret

In [22]:
# Add code to the following function so that it takes in two datasets, loops over one of them, and finds
# information about the nearest neighbors in the other dataset, based on a flag. 

def nncdist(data0,data1,r=None,nneighbors=None):
    
    ret = -1
    ret_list=[]
    if r is not None and nneighbors is not None:
        exit(-1)
        return ret
    elif r is not None and nneighbors is None:
        #combined= np.vstack((data0[0],data0[1])).T  
        #combined1= np.vstack((data1[0],data1[1])).T
        combined = data0.transpose()
        combined1 = data1.transpose()
        #print data0.transpose()
        dist=distance.cdist(combined,combined1,'euclidean')
        #print dist
        #print '******************************'
        for i,num in enumerate(dist):

            count=len(num[num<r])
            #count =len(dist[dist<r])
            ret_list.append(float(count)/(float(len(data1[0]))*r))
        ret_list = np.array(ret_list)
        return ret_list
    elif r is None and nneighbors is not None:
        for num0 in data0:
            diff = np.abs(num0 - data1)
            diff.sort()
            radius= diff[nneighbors-1]
            ret_list.append(1/radius)
        ret_list = np.array(ret_list)
        return ret_list
    return ret

In [23]:
def normal(x,mean,width):
    return (1.0/(width*np.sqrt(2*np.pi)))*(np.exp(-(x-mean)**2/(2*(width**2))))

# A product of two Gaussians
def signal_2D(npts,means,sigmas):
    pts = []
    for m,s in zip(means,sigmas):
        pts.append(np.random.normal(m,s,npts))
        
    pts = np.array(pts)
    return pts

# Flat in 2D
def background_2D(npts,lovals,hivals):
    pts = []
    for lo,hi in zip(lovals,hivals):
        width = hi-lo
        pts.append(lo + width*np.random.random(npts))
        
    pts = np.array(pts)
    return pts

# Helper function to generate signal and background at the same time
def gen_sig_and_bkg(npts,means,sigmas,lovals,hivals):
    sigpts = signal_2D(npts[0],means,sigmas)
    bkgpts = background_2D(npts[1],lovals,hivals)

    data = [sigpts[0].copy(),sigpts[1].copy()]
    data[0] = np.append(data[0],bkgpts[0])
    data[1] = np.append(data[1],bkgpts[1])
    data = np.array(data)
   
    return data

In [15]:
# Test the tools to generate the datasets.

sigpts = signal_2D(3000,[5.0,7.0],[0.1,0.1])
#sns.jointplot(sigpts[0],sigpts[1],kind='hex')

bkgpts = background_2D(10000,[3.5,5],[6,9])
#sns.jointplot(bkgpts[0],bkgpts[1],kind='hex')

data1 = [sigpts[0].copy(),sigpts[1].copy()]
data1[0] = np.append(data1[0],bkgpts[0])
data1[1] = np.append(data1[1],bkgpts[1])
data1 = np.array(data1)

#sns.jointplot(data[0],data[1],kind='hex')

In [24]:
nbkg = 10
sigmeans = [5.0,7.0]
sigwidths = [0.2,0.2]
bkglos = [3.5,5]
bkghis = [6,9]

def calc_pull(iterations, nsig, nMC, rad,cdist_bool):

    pull_frac_list=[]
    average_best_frac = 0
    frac = []
    fit_frac = []
    fit_frac_uncert = []
    
    frac_org = nsig/float(nsig+nbkg)
    

    for num in range(iterations):
        nsig_iteration = np.random.poisson(nsig)
        nbkg_iteration = np.random.poisson(nbkg)
        #nbkg_iteration = nbkg
        
        data = gen_sig_and_bkg([nsig_iteration,nbkg_iteration],sigmeans,sigwidths,bkglos,bkghis)
        #sns.jointplot(data[0],data[1],kind='hex')
        
        signal_points= signal_2D(nMC,sigmeans,sigwidths)
        background_points = background_2D(nMC,bkglos,bkghis)
        
        frac_iteration = float(nsig_iteration)/(float(nbkg_iteration+nsig_iteration))
        frac.append(frac_iteration)
        
        def probability(frac):
            tot_prob=[]
            ##################################################################
            # In this section, I'm using the analytic functions.
            # But we want to compare this to when we use the nearest neighbors approach
            #signal_prob =  normal(data[0],sigmeans[0],sigwidths[0])
            #signal_prob *= normal(data[1],sigmeans[1],sigwidths[1])

            #background_prob  = (1.0/(bkghis[0]-bkglos[0]))*np.ones(len(data[0]))
            #background_prob *= (1.0/(bkghis[1]-bkglos[1]))
            ##################################################################
            #####################NEAREST NEIGHBORS############################
            signal_prob=nn(data,signal_points, r=rad)
            #print 'not cdist',len(signal_prob[signal_prob<=0])
            background_prob= nn(data,background_points, r=rad)
            #print 'not cdist',len(background_prob[background_prob>0])
            #print len(background_prob[background_prob<=0])

            tot_prob.append(frac*signal_prob+ ((1-frac)*background_prob))
            #print tot_prob
            tot_prob=np.array(tot_prob)
            #tot_prob[tot_prob<=0.0] = 1e-64
            #print len(tot_prob[tot_prob<=0])
            
            return -np.log(tot_prob[tot_prob>0]).sum()
            #return -np.log(tot_prob).sum()
        
        def probability_cdist(frac):
            tot_prob=[]
            signal_prob=nncdist(data,signal_points, r=rad)
            #print 'cdist',len(signal_prob[signal_prob<=0])
            background_prob= nncdist(data,background_points, r=rad)
            #print 'cdist',len(background_prob[background_prob>0])
            tot_prob.append(frac*signal_prob+ ((1-frac)*background_prob))
            tot_prob=np.array(tot_prob)
            return -np.log(tot_prob[tot_prob>0]).sum()
            #return -np.log(tot_prob).sum()
        
        if cdist_bool:
            m1=Minuit(probability_cdist, frac= 0.05, limit_frac=(0.001,1), error_frac=0.001,  errordef = 0.5, print_level=0)
            m1.migrad()
            #m.hesse()

            if (m1.get_fmin().is_valid):
                param=m1.values
                err=m1.errors
                fit_frac.append(param["frac"])
                fit_frac_uncert.append(err["frac"])

                #print frac_org,frac_iteration,param["frac"],err["frac"]
                pull_frac=(frac_org-param["frac"])/err["frac"]
                pull_frac_list.append(pull_frac)
                
                
                
        else:
            m=Minuit(probability, frac= 0.05, limit_frac=(0.001,1), error_frac=0.001,  errordef = 0.5, print_level=0)
            m.migrad()
            #m.hesse()

            if (m.get_fmin().is_valid):
                param=m.values
                err=m.errors
                fit_frac.append(param["frac"])
                fit_frac_uncert.append(err["frac"])

                #print frac_org,frac_iteration,param["frac"],err["frac"]
                pull_frac=(frac_org-param["frac"])/err["frac"]
                pull_frac_list.append(pull_frac)
            
    return pull_frac_list, frac, fit_frac, fit_frac_uncert,iterations

In [27]:
means=[]
stds=[]
means_cdist=[]
stds_cdist=[]

for sig in [10]:
    for MC in [10]:
        start = time.time()
        for rad in [0.05]: #, 0.10,0.20,0.50]:
            start = time.time()
            print 'number of signal points: %s \t number of MC point: %s \tradius: %s' % (sig,MC,rad)
            pulls,org_vals,fit_vals,fit_uncerts,iteration = calc_pull(1, sig, MC, rad,False)
            

            parameters_mean = {'signal': sig, 'MC_points': MC, 'radius': rad, 'mean pulls': np.mean(pulls)}
            means.append(parameters_mean)
            parameters_std = {'signal': sig, 'MC_points': MC, 'radius': rad, 'mean stds': np.std(pulls)}
            stds.append(parameters_std)
            print "time to calc 10 iterations: %f s" % (time.time() - start)
            
            start=time.time()
            pulls_cdist,org_vals_cdist,fit_vals_cdist,fit_uncerts_cdist,iteration_cdist = calc_pull(1, sig, MC, rad,True)
            parameters_mean_cdist = {'signal': sig, 'MC_points': MC, 'radius': rad, 'mean pulls': np.mean(pulls_cdist)}
            means_cdist.append(parameters_mean_cdist)
            parameters_std_cdist = {'signal': sig, 'MC_points': MC, 'radius': rad, 'mean stds': np.std(pulls_cdist)}
            stds_cdist.append(parameters_std_cdist)
            print "time to calc 10 iterations: %f s" % (time.time() - start)
print means
print stds
print '--------------------'
print means_cdist
print stds_cdist




#WITH THE POISSON and 100 iterations- TESTING IT

number of signal points: 10 	 number of MC point: 10 	radius: 0.05
[[ 4.83438387  4.99333305  5.05711102  5.29889941  5.3110963   5.0043106
   5.27063044  3.57086568  4.96709845  3.56867275  3.62927788  5.87182299
   5.47726316  4.2129543   4.82001472  4.93990026]
 [ 7.32532698  7.09504552  6.86808311  6.72416714  7.16830591  6.74171561
   7.11955529  5.72152902  6.72977951  8.24147993  8.8330719   6.36507605
   6.85777888  6.8963387   8.21312815  6.51356875]]
-----------
[[ 4.83438387  4.99333305  5.05711102  5.29889941  5.3110963   5.0043106
   5.27063044  3.57086568  4.96709845  3.56867275  3.62927788  5.87182299
   5.47726316  4.2129543   4.82001472  4.93990026]
 [ 7.32532698  7.09504552  6.86808311  6.72416714  7.16830591  6.74171561
   7.11955529  5.72152902  6.72977951  8.24147993  8.8330719   6.36507605
   6.85777888  6.8963387   8.21312815  6.51356875]]
-----------
[[ 4.83438387  4.99333305  5.05711102  5.29889941  5.3110963   5.0043106
   5.27063044  3.57086568  4.96709845  3

In [ ]:
d0 = np.array([[1],[1]])
d1 = np.array([[3],[3]])

print nn(d0,d1,r=10)

In [ ]:
data = gen_sig_and_bkg([300,1000],sigmeans,sigwidths,bkglos,bkghis)
sns.jointplot(data[0],data[1],kind='hex')

#signal_points= signal_2D(10000,sigmeans,sigwidths)
background_points = background_2D(10000,bkglos,bkghis)
sns.jointplot(background_points[0],background_points[1],kind='hex')


In [ ]:
background_prob= nn(data,background_points, r=0.05)
print background_prob
print background_prob[background_prob<=0]